In [12]:
import os
from sklearn.datasets import load_iris
import pandas as pd

# Create indata directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Load iris dataset
iris = load_iris(as_frame=True)
df = iris.frame

# Save to CSV
df.to_csv('../data/iris.csv', index=False)

df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [13]:
from sklearn.preprocessing import StandardScaler

# Handle missing values (if any)
df = df.dropna()

# Encode the target column as categorical (if needed)
df['target'] = df['target'].astype('category')

# Feature scaling (standardization)

features = df.columns[:-1]  # all columns except 'target'
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Show the preprocessed dataframe
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,-0.900681,1.019004,-1.340227,-1.315444,0
1,-1.143017,-0.131979,-1.340227,-1.315444,0
2,-1.385353,0.328414,-1.397064,-1.315444,0
3,-1.506521,0.098217,-1.283389,-1.315444,0
4,-1.021849,1.249201,-1.340227,-1.315444,0


In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import mlflow.sklearn

mlflow.set_tracking_uri("http://146.56.165.194:5000")  # Set your MLflow tracking URI

# Prepare data
X = df[features]
y = df['target'].cat.codes  # convert categorical to int for sklearn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define experiments
experiments = {
    "LogisticRegression": LogisticRegression(max_iter=20, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in experiments.items():
    with mlflow.start_run(run_name=name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')
        
        # Log parameters, metrics, and model
        mlflow.log_params(model.get_params())
        mlflow.log_metrics({"accuracy": acc, "recall": recall, "f1_score": f1})
        mlflow.sklearn.log_model(model, name + "_model")

        mlflow.sklearn.save_model(model, "../models/"+ name + "_model")
        
        print(f"{name} - Accuracy: {acc:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")
        

2025/08/10 11:27:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 11:27:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://146.56.165.194:5000/#/experiments/0/runs/4d782807d94f429dbfff4f63edc31a90
🧪 View experiment at: http://146.56.165.194:5000/#/experiments/0


KeyboardInterrupt: 